<a href="https://colab.research.google.com/github/Prashanth1103/dsa/blob/main/Intro_to_AI_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras
!pip install scikit-learn
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 40.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
orbax-checkpoint 0.2.1 requires jax>=0.4.8, but you have jax 0.3.25 which is incompatible.
flax 0.6.9 requires jax>=0.4.2, but you have jax 0.3.25 which is incompatible.
chex 0.1.7 requires jax>=0.4.6, but you have jax 0.3.25 which is incompatible.


In [3]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
data = pd.read_pickle(r'/content/drive/MyDrive/merged_training.pkl')


In [5]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

train_text = train_data['text']
train_emotion = train_data['emotions']
test_text = test_data['text']
test_emotion = test_data['emotions']

train_emotion[train_emotion == 'sadness'] = 0
train_emotion[train_emotion == 'joy'] = 1
train_emotion[train_emotion == 'love'] = 2
train_emotion[train_emotion == 'anger'] = 3
train_emotion[train_emotion == 'fear'] = 4
train_emotion[train_emotion == 'surprise'] = 5

test_emotion[test_emotion == 'sadness'] = 0
test_emotion[test_emotion == 'joy'] = 1
test_emotion[test_emotion == 'love'] = 2
test_emotion[test_emotion == 'anger'] = 3
test_emotion[test_emotion == 'fear'] = 4
test_emotion[test_emotion == 'surprise'] = 5

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_text)
train_sequences = tokenizer.texts_to_sequences(train_text)
test_sequences = tokenizer.texts_to_sequences(test_text)

max_length = max([len(i) for i in train_sequences])
train_padded = pad_sequences(train_sequences, maxlen=max_length)
test_padded = pad_sequences(test_sequences, maxlen=max_length)

train_labels = to_categorical(train_emotion)
test_labels = to_categorical(test_emotion)

In [7]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, 100, input_length=max_length))
model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(train_padded, train_labels, epochs=3, batch_size=320, validation_data=(test_padded, test_labels))

accuracy = model.evaluate(test_padded, test_labels)[1]
print("Accuracy:", accuracy)

Epoch 1/3
1043/1043 [==============================] - 808s 775ms/step - loss: 0.3176 - accuracy: 0.8671 - val_loss: 0.0994 - val_accuracy: 0.9367
Epoch 2/3
1043/1043 [==============================] - 756s 725ms/step - loss: 0.0978 - accuracy: 0.9390 - val_loss: 0.0947 - val_accuracy: 0.9346
Epoch 3/3
2606/2606 [==============================] - 140s 54ms/step - loss: 0.0979 - accuracy: 0.9371
Accuracy: 0.9371296167373657


In [9]:
model.save("/content/drive/MyDrive/lstm_model.h5")

In [22]:
from keras.models import load_model

In [24]:
model = load_model('/content/drive/MyDrive/lstm_model.h5')

In [15]:
input_text = "I am scared whether this will work out well"
input_text2 = pd.DataFrame([input_text])
input_vector = tokenizer.texts_to_sequences(input_text2[0])
input_pad_sequence = pad_sequences(input_vector, maxlen=max_length)
prediction = model.predict(input_pad_sequence, verbose = 'auto')


1/1 [==============================] - 0s 68ms/step


In [16]:
import numpy
predicted_class = numpy.argmax(prediction)
emotion_list = ['sad', 'happy', 'love', 'anger', 'fear', 'surprise']
emotion_list[predicted_class]

'fear'